Setting up a connection to the Database

In [94]:
# Connection parameters
server = 'ANUBHAV-GALAXY-\SQLEXPRESS'  
database = 'AdventureWorks2019'  
driver = 'ODBC Driver 17 for SQL Server'  # Ensure you have this driver installed
trusted_connection = 'yes'

from sqlalchemy import create_engine
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI



# Create SQLAlchemy engine
connection_string = f"mssql+pyodbc:///?odbc_connect=DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection={trusted_connection};"
engine = create_engine(connection_string)

# Create SQLDatabase object 
db = SQLDatabase(engine,schema = 'Production')



<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Anubhav\AppData\Local\Temp\ipykernel_96152\2908002570.py:2: SyntaxWarning: invalid escape sequence '\S'
  server = 'ANUBHAV-GALAXY-\SQLEXPRESS'
c:\Users\Anubhav\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'hierarchyid' of column 'DocumentNode'
  self._metadata.reflect(
c:\Users\Anubhav\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  self._metadata.reflect(


Checking out the connection and what all things it can do for us

In [173]:
# Checking if the engine works
# Running a sample query

db.run("SELECT TOP 5 Name,ProductCategoryID FROM [Production].[ProductCategory] ORDER BY [ProductCategoryID]")


"[('Bikes', 1), ('Components', 2), ('Clothing', 3), ('Accessories', 4)]"

In [177]:
# Some imp methods of the SQLDatabase Object

print(db.get_table_info())
#db.get_table_names()
# db.dialect


CREATE TABLE [Production].[BillOfMaterials] (
	[BillOfMaterialsID] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductAssemblyID] INTEGER NULL, 
	[ComponentID] INTEGER NOT NULL, 
	[StartDate] DATETIME NOT NULL DEFAULT (getdate()), 
	[EndDate] DATETIME NULL, 
	[UnitMeasureCode] NCHAR(3) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[BOMLevel] SMALLINT NOT NULL, 
	[PerAssemblyQty] DECIMAL(8, 2) NOT NULL DEFAULT ((1.00)), 
	[ModifiedDate] DATETIME NOT NULL DEFAULT (getdate()), 
	CONSTRAINT [PK_BillOfMaterials_BillOfMaterialsID] PRIMARY KEY ([BillOfMaterialsID]), 
	CONSTRAINT [FK_BillOfMaterials_Product_ComponentID] FOREIGN KEY([ComponentID]) REFERENCES [Production].[Product] ([ProductID]), 
	CONSTRAINT [FK_BillOfMaterials_Product_ProductAssemblyID] FOREIGN KEY([ProductAssemblyID]) REFERENCES [Production].[Product] ([ProductID]), 
	CONSTRAINT [FK_BillOfMaterials_UnitMeasure_UnitMeasureCode] FOREIGN KEY([UnitMeasureCode]) REFERENCES [Production].[UnitMeasure] ([UnitMeasureCode])
)

/*
3 rows fro

Now we can start off with the chain components and diving into them

Create_SQL_Query_Chain

In [183]:
from langchain.chains import create_sql_query_chain

llm = ChatOpenAI(model = 'gpt-4')

# Instantiating the create_sql_query_chain
writer_query = create_sql_query_chain(llm = llm, db = db)

# Invoking the chain
response = writer_query.invoke({"question": "How many countries are there in the world?",'table_names_to_use':['ProductCategory']})
print(response)

I'm afraid I can't answer your question using the available tables. The tables provided do not contain any data about countries.


In [184]:
# Checking out the underlying prompt
from langchain.chains.sql_database.prompt import PROMPT, SQL_PROMPTS

print(PROMPT.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}


In [150]:
from langchain_core.prompts import PromptTemplate

prompt_template = """You are a helpful assistant tasked with generating plain SQL queries based on user questions.

Guidelines:
1. Generate a syntactically correct {dialect} SQL query that answers the user's question.
2. Provide only the SQL query as the output. 
   - Do not include any markdown formatting (e.g., ```sql or ```)
   - Do not add prefixes like "SQLQuery:" or any other additional text.
3. Limit the query to {top_k} results unless the user specifies otherwise.
4. Order results by a relevant column, if applicable, to return the most interesting examples.
5. Never select all columns; include only those relevant to the question.
6. Use only the column names from the schema provided. Avoid querying non-existent columns.
7. **If the user's question cannot be answered using the provided schema or data, respond with: "We don't have related data in the database".**

### Example:
User's question: "How many product categories are there?"
Response:
SELECT COUNT([ProductCategoryID]) AS [CategoryCount]
FROM [Production].[ProductCategory]

Notice:
- The response contains only the plain SQL query without prefixes or suffixes.
- No markdown or additional formatting.

Only use the following tables:
{table_info}

Question: {input}"""
prompt = PromptTemplate.from_template(prompt_template)



In [187]:
# Executing with the customized prompt

response = writer_query.invoke({"question": "List the product categories ",
                         'table_names_to_use': ['ProductCategory'],
                         'prompt':prompt})

print(response)

SELECT TOP 5 [Name] FROM [Production].[ProductCategory]


Execute Query component

In [191]:
# A tool to execute sql queries using the engine
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool


# Instantiating the tool
execute_query = QuerySQLDataBaseTool(db=db)

# Chain
sql_chain = writer_query | execute_query
response = sql_chain.invoke({"question": "How many categories in product exist",
                         "table_names_to_use":['ProductCategory']})

In [192]:
response

'[(4,)]'

Answer Generator

In [193]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=writer_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [195]:
chain.invoke({"question": "How many categories in product exist",
              "table_names_to_use":['ProductCategory']})

'There are 4 product categories.'

In [ ]:
{"question": "How many categories in product exist",
              "table_names_to_use":['ProductCategory'],
              'query':<'sql query'>,
              'result':<'info from db'}